In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
HIST_PATH = "historical_data.csv"
FG_PATH   = "fear_greed_index.csv"
OUT_DIR   = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)


In [3]:
def normalize_cols(df):
    # make column names simple: lowercase, underscores
    df = df.copy()
    newcols = {}
    for c in df.columns:
        cn = c.strip().lower().replace(' ', '_').replace('.', '').replace('/', '_')
        newcols[c] = cn
    return df.rename(columns=newcols)

def detect_time_series_units(series):
    # returns 'ms', 's' or None
    vals = series.dropna().astype(float)
    if len(vals) == 0:
        return None
    med = np.median(vals)
    # heuristics:
    if med > 1e12:
        return 'ms'
    if med > 1e9:
        return 's'
    return None

In [4]:
hist = pd.read_csv(HIST_PATH)
fg   = pd.read_csv(FG_PATH)

hist = normalize_cols(hist)
fg   = normalize_cols(fg)

print("Historical columns:", hist.columns.tolist())
print("FG columns:", fg.columns.tolist())

# Save initial samples
hist.head(5).to_csv(os.path.join(OUT_DIR,"hist_head.csv"), index=False)
fg.head(5).to_csv(os.path.join(OUT_DIR,"fg_head.csv"), index=False)

Historical columns: ['account', 'coin', 'execution_price', 'size_tokens', 'size_usd', 'side', 'timestamp_ist', 'start_position', 'direction', 'closed_pnl', 'transaction_hash', 'order_id', 'crossed', 'fee', 'trade_id', 'timestamp']
FG columns: ['timestamp', 'value', 'classification', 'date']


In [7]:
# Historical timestamp: common names 'timestamp', 'time', 'date', 'datetime'
hist_time_col = None
for c in ['timestamp','time','date','datetime']:
    if c in hist.columns:
        hist_time_col = c
        break



In [9]:
# FG: based on your file the date column is 'date' (string like '2018-02-01')
fg_date_col = 'date' if 'date' in fg.columns else None

# Parse HIST timestamp: detect ms/s or ISO
if hist_time_col:
    unit = detect_time_series_units(hist[hist_time_col])
    if unit == 'ms':
        hist['parsed_time'] = pd.to_datetime(hist[hist_time_col].astype(float), unit='ms', errors='coerce')
        print("Parsed historical timestamps as milliseconds (unit='ms').")
    elif unit == 's':
        hist['parsed_time'] = pd.to_datetime(hist[hist_time_col].astype(float), unit='s', errors='coerce')
        print("Parsed historical timestamps as seconds (unit='s').")
    else:
        # fallback: try direct parse (ISO or already readable)
        hist['parsed_time'] = pd.to_datetime(hist[hist_time_col], errors='coerce')
        print("Parsed historical timestamps using pd.to_datetime (fallback).")
else:
    hist['parsed_time'] = pd.NaT
    print("No timestamp-like column detected in historical data; 'parsed_time' is NaT.")

# Parse FG date column directly as a calendar date string (FIXED)
if fg_date_col:
    fg['parsed_date'] = pd.to_datetime(fg[fg_date_col], errors='coerce')   # <-- SIMPLE direct parse (no astype(float))
    print("Parsed FG 'date' column using pd.to_datetime.")
else:
    fg['parsed_date'] = pd.NaT
    print("No FG date column detected; 'parsed_date' is NaT.")

# Create date keys for daily join
hist['trade_date'] = hist['parsed_time'].dt.date
fg['fg_date'] = fg['parsed_date'].dt.date

print("Historical time span:", hist['parsed_time'].min(), "to", hist['parsed_time'].max())
print("FG time span:", fg['parsed_date'].min(), "to", fg['parsed_date'].max())


Parsed historical timestamps as milliseconds (unit='ms').
Parsed FG 'date' column using pd.to_datetime.
Historical time span: 2023-03-28 10:40:00 to 2025-06-15 15:06:40
FG time span: 2018-02-01 00:00:00 to 2025-05-02 00:00:00


In [10]:
def pick(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

pnl_col    = pick(hist.columns, ['closed_pnl','closedpnl','pnl','profit','realized_pnl','realised_pnl','closed_pnl_usd'])
size_col   = pick(hist.columns, ['size_usd','size','size_tokens','size_usd...on_hash','size_usd'])
price_col  = pick(hist.columns, ['execution_price','executionprice','price'])
side_col   = pick(hist.columns, ['side','direction'])
account_col= pick(hist.columns, ['account','user','client'])
symbol_col = pick(hist.columns, ['coin','symbol','pair'])
leverage_col = pick(hist.columns, ['leverage','lev'])

print("Detected mapping:")
print("pnl:", pnl_col, "size:", size_col, "price:", price_col, "side:", side_col)
print("account:", account_col, "symbol:", symbol_col, "leverage:", leverage_col)


Detected mapping:
pnl: closed_pnl size: size_usd price: execution_price side: side
account: account symbol: coin leverage: None


In [11]:
# FG: if 'value' numeric exists use it; else map classification strings to ordinal
if 'value' in fg.columns:
    fg['fg_index'] = pd.to_numeric(fg['value'], errors='coerce')
else:
    # map common labels ("Extreme Fear", "Fear", "Neutral", "Greed", "Extreme Greed") to numeric
    if 'classification' in fg.columns:
        def map_label(x):
            s = str(x).lower()
            if 'extreme fear' in s: return 10.0
            if 'fear' in s: return 25.0
            if 'neutral' in s: return 50.0
            if 'extreme greed' in s: return 90.0
            if 'greed' in s: return 75.0
            try:
                return float(x)
            except:
                return np.nan
        fg['fg_index'] = fg['classification'].apply(map_label)
    else:
        fg['fg_index'] = np.nan

# Merge hist (trades) with FG on calendar date (left join)
fg_merge_cols = ['fg_date']
if 'classification' in fg.columns:
    fg_merge_cols.append('classification')
fg_merge_cols.append('fg_index')

fg_small = fg[fg_merge_cols].drop_duplicates()
merged = hist.merge(fg_small, left_on='trade_date', right_on='fg_date', how='left')
print("Merged shape:", merged.shape)

# Save merged quick
merged.to_csv(os.path.join(OUT_DIR, "cleaned_merged_raw.csv"), index=False)

Merged shape: (211224, 21)


In [12]:
# Closed PnL numeric
if pnl_col and pnl_col in merged.columns:
    merged['closed_pnl'] = pd.to_numeric(merged[pnl_col], errors='coerce')
else:
    merged['closed_pnl'] = np.nan
    print("Warning: closed PnL column not found; 'closed_pnl' set to NaN. If you have a PnL column, rename it to one of:",
          "closed_pnl, closedpnl, pnl, profit, realized_pnl, realised_pnl")

# Notional exposure (prefer size_usd)
if 'size_usd' in merged.columns and merged['size_usd'].notna().sum() > 0:
    merged['notional'] = pd.to_numeric(merged['size_usd'], errors='coerce')
elif size_col and price_col and size_col in merged.columns and price_col in merged.columns:
    merged['notional'] = pd.to_numeric(merged[size_col], errors='coerce') * pd.to_numeric(merged[price_col], errors='coerce')
else:
    merged['notional'] = np.nan

merged['pnl_per_notional'] = merged['closed_pnl'] / merged['notional']
merged['is_win'] = merged['closed_pnl'] > 0

if leverage_col and leverage_col in merged.columns:
    merged['leverage'] = pd.to_numeric(merged[leverage_col], errors='coerce')
else:
    merged['leverage'] = np.nan

if side_col and side_col in merged.columns:
    merged['side_norm'] = merged[side_col].astype(str).str.lower()
else:
    merged['side_norm'] = np.nan

# Save cleaned merged
cleaned_path = os.path.join(OUT_DIR, "cleaned_merged_final.csv")
merged.to_csv(cleaned_path, index=False)
print("Saved cleaned merged to:", cleaned_path)


Saved cleaned merged to: outputs/cleaned_merged_final.csv


In [13]:
# Closed PnL numeric
if pnl_col and pnl_col in merged.columns:
    merged['closed_pnl'] = pd.to_numeric(merged[pnl_col], errors='coerce')
else:
    merged['closed_pnl'] = np.nan
    print("Warning: closed PnL column not found; 'closed_pnl' set to NaN. If you have a PnL column, rename it to one of:",
          "closed_pnl, closedpnl, pnl, profit, realized_pnl, realised_pnl")

# Notional exposure (prefer size_usd)
if 'size_usd' in merged.columns and merged['size_usd'].notna().sum() > 0:
    merged['notional'] = pd.to_numeric(merged['size_usd'], errors='coerce')
elif size_col and price_col and size_col in merged.columns and price_col in merged.columns:
    merged['notional'] = pd.to_numeric(merged[size_col], errors='coerce') * pd.to_numeric(merged[price_col], errors='coerce')
else:
    merged['notional'] = np.nan

merged['pnl_per_notional'] = merged['closed_pnl'] / merged['notional']
merged['is_win'] = merged['closed_pnl'] > 0

if leverage_col and leverage_col in merged.columns:
    merged['leverage'] = pd.to_numeric(merged[leverage_col], errors='coerce')
else:
    merged['leverage'] = np.nan

if side_col and side_col in merged.columns:
    merged['side_norm'] = merged[side_col].astype(str).str.lower()
else:
    merged['side_norm'] = np.nan

# Save cleaned merged
cleaned_path = os.path.join(OUT_DIR, "cleaned_merged_final.csv")
merged.to_csv(cleaned_path, index=False)
print("Saved cleaned merged to:", cleaned_path)

# ----------------------------
# 6) Exploratory Data Analysis (plots + tables)
# ----------------------------
print("\n=== EDA Summary ===")
total_trades = len(merged)
pnl_nonnull = merged['closed_pnl'].notna().sum()
unique_accounts = merged[account_col].nunique() if account_col in merged.columns else None
unique_symbols = merged[symbol_col].nunique() if symbol_col in merged.columns else None
print("Total trades:", total_trades)
print("Trades with closed_pnl:", pnl_nonnull)
print("Unique accounts:", unique_accounts)
print("Unique symbols:", unique_symbols)

# Descriptive stats
desc = merged[['closed_pnl','notional','pnl_per_notional','leverage']].describe()
desc.to_csv(os.path.join(OUT_DIR, "descriptive_stats.csv"))
print("Saved descriptive stats.")

# ---- Plot: daily closed PnL time series (and FG index if present) ----
daily = merged.groupby('trade_date').agg(
    daily_closed_pnl=('closed_pnl','sum'),
    trade_count=('closed_pnl','count')
).reset_index()

if 'fg_index' in merged.columns:
    fg_by_date = merged[['fg_date','fg_index']].drop_duplicates().dropna(subset=['fg_date']).sort_values('fg_date')
    daily = daily.merge(fg_by_date, left_on='trade_date', right_on='fg_date', how='left')

plt.figure(figsize=(12,4))
plt.plot(pd.to_datetime(daily['trade_date']), daily['daily_closed_pnl'])
plt.title("Daily aggregated closed PnL")
plt.xlabel("Date")
plt.ylabel("Daily closed PnL (sum)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,"daily_closed_pnl_timeseries.png"))
plt.close()

# If FG index exists, overlay on second axis
if 'fg_index' in daily.columns:
    fig, ax1 = plt.subplots(figsize=(12,4))
    ax1.plot(pd.to_datetime(daily['trade_date']), daily['daily_closed_pnl'], label='Daily PnL')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Daily PnL')
    ax2 = ax1.twinx()
    ax2.plot(pd.to_datetime(daily['trade_date']), daily['fg_index'], linestyle='--', label='FG index')
    ax2.set_ylabel('Fear-Greed Index')
    plt.title("Daily closed PnL and Fear-Greed Index")
    fig.tight_layout()
    fig.savefig(os.path.join(OUT_DIR,"daily_pnl_fgindex.png"))
    plt.close()

# ---- Histogram of per-trade closed_pnl ----
plt.figure(figsize=(8,4))
merged['closed_pnl'].dropna().hist(bins=100)
plt.title("Histogram of closed_pnl (per trade)")
plt.xlabel("closed_pnl")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,"hist_closed_pnl.png"))
plt.close()

# ---- Boxplot: closed_pnl by sentiment classification (top categories) ----
if 'classification' in merged.columns:
    top_cats = merged['classification'].value_counts().head(8).index.tolist()
    data = [merged.loc[merged['classification'] == cat, 'closed_pnl'].dropna().values for cat in top_cats]
    # filter out empty groups
    data_nonempty = [d for d in data if len(d) > 0]
    labels_nonempty = [lab for lab, d in zip(top_cats, data) if len(d) > 0]
    if len(data_nonempty) > 0:
        plt.figure(figsize=(12,5))
        plt.boxplot(data_nonempty, labels=labels_nonempty, showfliers=False)
        plt.title("closed_pnl distribution by sentiment classification (top categories)")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR,"boxplot_pnl_by_sentiment.png"))
        plt.close()

# ---- Heatmap: average PnL by leverage bucket vs sentiment ----
if 'leverage' in merged.columns and 'classification' in merged.columns:
    tmp = merged.copy()
    tmp['lev_bucket'] = pd.cut(tmp['leverage'].fillna(-1), bins=[-1,0,2,5,10,20,1e9], labels=['NA','0-2','2-5','5-10','10-20','20+'])
    pivot = tmp.groupby(['lev_bucket','classification'])['closed_pnl'].mean().unstack(fill_value=np.nan)
    if pivot.shape[0] > 0 and pivot.shape[1] > 0:
        plt.figure(figsize=(10,4))
        plt.imshow(pivot.fillna(0).values, aspect='auto')
        plt.title("Avg closed_pnl by leverage bucket × sentiment")
        plt.xticks(range(pivot.shape[1]), pivot.columns, rotation=45, ha='right')
        plt.yticks(range(pivot.shape[0]), pivot.index)
        plt.colorbar()
        plt.tight_layout()
        plt.savefig(os.path.join(OUT_DIR,"heatmap_lev_sentiment.png"))
        plt.close()

# Save a sample view
cols_for_sample = ['trade_date','parsed_time','account','coin','closed_pnl','notional','classification','fg_index']
sample = merged[cols_for_sample].head(200)
sample.to_csv(os.path.join(OUT_DIR,"merged_sample_200.csv"), index=False)

Saved cleaned merged to: outputs/cleaned_merged_final.csv

=== EDA Summary ===
Total trades: 211224
Trades with closed_pnl: 211224
Unique accounts: 32
Unique symbols: 246
Saved descriptive stats.


/tmp/ipython-input-2741620310.py:105: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data_nonempty, labels=labels_nonempty, showfliers=False)
/tmp/ipython-input-2741620310.py:116: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pivot = tmp.groupby(['lev_bucket','classification'])['closed_pnl'].mean().unstack(fill_value=np.nan)


In [15]:
test_results = {}
if 'classification' in merged.columns:
    cl = merged['classification'].astype(str).str.lower()
    fear = merged.loc[cl.str.contains('fear', na=False), 'closed_pnl'].dropna()
    greed = merged.loc[cl.str.contains('greed', na=False), 'closed_pnl'].dropna()
    test_results['n_fear'] = int(len(fear))
    test_results['n_greed'] = int(len(greed))
    print("Counts: fear", test_results['n_fear'], "greed", test_results['n_greed'])
    if len(fear) >= 30 and len(greed) >= 30:
        tstat, pval = stats.ttest_ind(fear, greed, equal_var=False, nan_policy='omit')
        test_results['ttest'] = {'tstat': float(tstat), 'pval': float(pval)}
        print("T-test Fear vs Greed: tstat={:.4f}, p={:.4g}".format(tstat, pval))
    else:
        test_results['ttest'] = None
        print("Not enough observations for robust t-test (need >=30 each).")
else:
    print("No classification column for statistical test.")

with open(os.path.join(OUT_DIR, "stat_tests.txt"), "w") as f:
    f.write(str(test_results))

# ----------------------------
# 8) Regression: effect of FG index on closed PnL (controlled)
# ----------------------------
REG_SAMPLE_MAX = 20000
reg_cols = ['closed_pnl','fg_index','leverage','side_norm']
if symbol_col and symbol_col in merged.columns:
    reg_cols.append(symbol_col)
if account_col and account_col in merged.columns:
    reg_cols.append(account_col)

reg_df = merged[[c for c in reg_cols if c in merged.columns]].copy().dropna(subset=['closed_pnl','fg_index'])
print("Rows available for regression:", len(reg_df))
if len(reg_df) > REG_SAMPLE_MAX:
    reg_df = reg_df.sample(n=REG_SAMPLE_MAX, random_state=42)

if len(reg_df) >= 50:
    # reduce dimensionality for categorical variables by keeping top categories
    if account_col in reg_df.columns:
        top_accounts = reg_df[account_col].value_counts().head(30).index.tolist()
        reg_df['account2'] = reg_df[account_col].where(reg_df[account_col].isin(top_accounts), other='OTHER_ACC')
    else:
        reg_df['account2'] = 'NOACC'

    if symbol_col in reg_df.columns:
        top_symbols = reg_df[symbol_col].value_counts().head(20).index.tolist()
        reg_df['symbol2'] = reg_df[symbol_col].where(reg_df[symbol_col].isin(top_symbols), other='OTHER_SYM')
    else:
        reg_df['symbol2'] = 'NOSYM'

    formula = "closed_pnl ~ fg_index + leverage + C(side_norm) + C(symbol2) + C(account2)"
    try:
        model = smf.ols(formula=formula, data=reg_df).fit(cov_type='HC3')
        with open(os.path.join(OUT_DIR, "regression_summary.txt"), "w") as f:
            f.write(model.summary().as_text())
        print("Regression completed. Coefficients (top):")
        print(model.params.head(10))
    except Exception as e:
        print("Regression error:", e)
else:
    print("Not enough rows for regression after dropna; skipping regression.")

# ----------------------------
# 9) Account-level aggregation & clustering
# ----------------------------
if account_col and account_col in merged.columns:
    acct = merged.groupby(account_col).agg(
        trades=('closed_pnl','count'),
        total_pnl=('closed_pnl','sum'),
        mean_pnl=('closed_pnl','mean'),
        win_rate=('is_win','mean'),
        avg_leverage=('leverage','mean'),
        avg_notional=('notional','mean')
    ).reset_index().fillna(0)
    acct.to_csv(os.path.join(OUT_DIR,"account_summary_raw.csv"), index=False)
    # Filter for accounts with >=10 trades to cluster
    acct_filt = acct[acct['trades'] >= 10].copy()
    print("Accounts with >=10 trades:", len(acct_filt))
    if len(acct_filt) >= 5:
        features = ['trades','total_pnl','mean_pnl','win_rate','avg_leverage']
        X = acct_filt[features].fillna(0).values
        Xs = StandardScaler().fit_transform(X)
        k = min(5, max(2, len(acct_filt)//20))
        kmeans = KMeans(n_clusters=k, random_state=42)
        acct_filt['cluster'] = kmeans.fit_predict(Xs)
        acct_filt.to_csv(os.path.join(OUT_DIR,"account_clusters.csv"), index=False)
        print("Saved account clusters to outputs.")
    else:
        print("Not enough accounts with >=10 trades for clustering.")
else:
    print("No account column found; skipping account-level aggregation & clustering.")



Counts: fear 133871 greed 43251
T-test Fear vs Greed: tstat=-4.9038, p=9.421e-07
Rows available for regression: 184263
Regression error: zero-size array to reduction operation maximum which has no identity
Accounts with >=10 trades: 32
Saved account clusters to outputs.


In [16]:
report_lines = []
report_lines.append("=== Quick Summary ===")
report_lines.append(f"Total trades: {total_trades}")
report_lines.append(f"Trades with closed_pnl: {pnl_nonnull}")
if 'classification' in merged.columns:
    report_lines.append("Sentiment counts:\n" + merged['classification'].value_counts().to_string())
if 'fg_index' in merged.columns:
    report_lines.append(f"FG index range: {merged['fg_index'].min()} to {merged['fg_index'].max()}")
report_lines.append(f"Saved outputs to folder: {OUT_DIR}")
with open(os.path.join(OUT_DIR,"quick_report.txt"), "w") as f:
    f.write("\n".join(report_lines))

print("\nPipeline finished. Check the 'outputs' folder for CSVs, PNGs and text summaries.")


Pipeline finished. Check the 'outputs' folder for CSVs, PNGs and text summaries.
